# Food Spoilage Kernel Case Study

## Setup

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

from lifetime_modeling import *
from ablation_modeling import *

In [2]:
load_execution_time_from_csv("timing-ablation.csv")
load_embodied_values_from_csv("embodied-carbon-ablation.csv")
load_memory_from_csv("memory-ablation.csv")

## Lifetime Modeling

In [ ]:
workload_dropdown = widgets.Dropdown(
    options=execution_time.keys(),
    value="DT-Small",
    description="Kernel:",
)

lifetime_slider = widgets.FloatSlider(
    value=1.0,
    min=MIN_LIFETIME,
    max=MAX_LIFETIME,
    step=0.5,
    description='Lifetime (yrs)'
)

inf_freq_slider = widgets.BoundedFloatText(
    value=1.0,
    min=MIN_INF_FREQ,
    max=MAX_INF_FREQ,
    step=1.0,
    description='Number of Executed Tasks per Year'
)

power_source_dropdown = widgets.Dropdown(
    options=carbon_intensity_values.keys(),
    value=list(carbon_intensity_values.keys())[0],
    description="Power Source:",
)

carbon_intensity_box = widgets.Text(
    value=carbon_intensity_values[power_source_dropdown.value],
    description="Carbon Int. (CO₂/unit energy)",
)

carbon_intensity_box.disabled = True
def update_carbon_intensity(*args):
    if power_source_dropdown.value == "Custom":
        carbon_intensity_box.disabled = False
    else:
        carbon_intensity_box.value = carbon_intensity_values[power_source_dropdown.value]
        carbon_intensity_box.disabled = True

power_source_dropdown.observe(update_carbon_intensity, 'value')

core_freq_box = widgets.BoundedFloatText(
    value=10000.0,
    min=1000.0,
    max=100000.0,
    step=100.0,
    description='Core Freq'
)

ui = widgets.VBox([
    workload_dropdown,
    lifetime_slider,
    inf_freq_slider,
    power_source_dropdown,
    carbon_intensity_box,
    core_freq_box
])

out = widgets.interactive_output(
    plot_carbon_vs_lifetime,
    {
        'workload_choice': workload_dropdown,
        'lifetime_yrs': lifetime_slider,
        'inf_freq': inf_freq_slider,
        'carbon_intensity': carbon_intensity_box,
        'core_freq': core_freq_box,
    }
)

display(ui, out)

Output()

## Carbon-Accuracy Kernel Modeling

In [ ]:

import matplotlib.pyplot as plt

def plot_accuracy_vs_carbon_with_fig(lifetime_years, tasks_per_year, carbon_intensity):
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_accuracy_vs_carbon(
        lifetime_years=lifetime_years,
        tasks_per_year=tasks_per_year,
        carbon_intensity=carbon_intensity,
        ax=ax
    )
    plt.show()

# Reasonable slider ranges for interactive exploration
interact(
    plot_accuracy_vs_carbon_with_fig,
    lifetime_years=FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Lifetime (years)'),
    tasks_per_year=IntSlider(value=365, min=1, max=10000, step=1, description='Tasks/year'),
    carbon_intensity=Dropdown(
        options=carbon_intensity_values,
        value=carbon_intensity_values["US Avg (2023)"],
        description='Carbon Intensity\n(gCO2e/Wh)'
    )
)


interactive(children=(FloatSlider(value=1.0, description='Lifetime (years)', max=10.0, min=0.1), IntSlider(val…

<function __main__.plot_accuracy_vs_carbon_with_fig(lifetime_years, tasks_per_year, carbon_intensity)>